In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
%matplotlib inline

### 처리시간을 출력해주는 함수 만들기

In [ ]:
def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### train data 가져오기

In [ ]:
train = pd.read_csv('../input/training/training.csv')

In [ ]:
train.info()

#### na 값 drop 하기

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.info()

In [ ]:
train.shape

#### Index 번호 다시 부여하기

In [ ]:
train.index = pd.RangeIndex(len(train.index))

In [ ]:
train.tail(3)

### test data 가져오기

In [ ]:
test = pd.read_csv('../input/test/test.csv')

In [ ]:
test.info()

In [ ]:
test.shape

### Image 배열화 하기

In [ ]:
len(train), len(test)

In [ ]:
len(train.Image[0].split(' ')), len(test.Image[0].split(' '))

In [ ]:
train.Image[0].split(' ')[0], test.Image[0].split(' ')[0]

In [ ]:
for x in range(len(train)):
    train.Image[x] = np.asarray(train.Image[x].split(' '), dtype=np.uint8).reshape(96, 96)

In [ ]:
for x in range(len(test)):
    test.Image[x] = np.asarray(test.Image[x].split(' '), dtype=np.uint8).reshape(96, 96)

In [ ]:
train.shape, test.shape

### y값 만들기

In [ ]:
y = train.iloc[:, :-1].values

In [ ]:
y.shape

In [ ]:
y[1,:]

### Key_points 찍어보는 함수 만들기

In [ ]:
plt.imshow(train.Image[0])
plt.show()

In [ ]:
def keypoints_show(x, y=None):
    plt.imshow(x, 'gray')
    if y is not None:
        points = np.vstack(np.split(y, 15)).T
        plt.plot(points[0], points[1], 'o', color='red')
    plt.axis('off')   

In [ ]:
sample_idx = np.random.choice(len(train))

In [ ]:
y[sample_idx]

In [ ]:
train.Image[sample_idx]

In [ ]:
keypoints_show(train.Image[sample_idx], y[sample_idx])

### train / test data shape 변경

In [ ]:
x = np.stack(train.Image)[..., None]

In [ ]:
x.shape

In [ ]:
x_t = np.stack(test.Image)[..., None]

In [ ]:
x_t.shape

### Data scaling

In [ ]:
x = x / 255.0

In [ ]:
x_t = x_t / 255.0

### Model

In [ ]:
from IPython.display import SVG
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, Flatten, LeakyReLU, ELU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.vis_utils import model_to_dot

#### create model 1 (CNN)

In [ ]:
# model = Sequential()

# model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'linear', input_shape = (96, 96, 1)))
# model.add(LeakyReLU(alpha=.001))
# model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.7))
# model.add(Flatten())
# model.add(Dense(256, activation = 'linear'))
# model.add(LeakyReLU(alpha=.001))
# model.add(Dropout(0.7))
# model.add(Dense(128, activation = 'linear'))
# model.add(LeakyReLU(alpha=.001))
# model.add(Dropout(0.7))
# model.add(Dense(30))

#### create model 2 (CNN)

In [ ]:
# model2 = Sequential()

# model2.add(Conv2D(filters = 64, kernel_size = (4,4), padding = 'Same', activation = 'linear', input_shape = (96, 96, 1)))
# model2.add(LeakyReLU(alpha=.001))
# model2.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model2.add(Dropout(0.5))
# model2.add(Flatten())
# model2.add(Dense(256, activation = 'linear'))
# model2.add(LeakyReLU(alpha=.001))
# model2.add(Dropout(0.7))
# model2.add(Dense(30))

#### create model 3 (CNN)

In [ ]:
# model3 = Sequential()

# model3.add(Conv2D(filters = 128, kernel_size = (5,5), padding = 'Same', activation = 'linear', input_shape = (96, 96, 1)))
# model3.add(LeakyReLU(alpha=.001))
# model3.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model3.add(Dropout(0.5))
# model3.add(Flatten())
# model3.add(Dense(256, activation = 'linear'))
# model3.add(LeakyReLU(alpha=.001))
# model3.add(Dropout(0.5))
# model3.add(Dense(128, activation = 'linear'))
# model3.add(LeakyReLU(alpha=.001))
# model3.add(Dropout(0.7))
# model3.add(Dense(30))

#### create model 4 (CNN) - ELU

In [ ]:
# np.random.seed(777)

# model4 = Sequential()

# model4.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'elu', input_shape = (96, 96, 1)))
# model4.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model4.add(Dropout(0.3))

# model4.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'elu'))
# model4.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model4.add(Dropout(0.5))

# model4.add(Flatten())
# model4.add(Dense(128, activation = 'relu'))
# model4.add(Dropout(0.5))
# model4.add(Dense(30, activation = 'linear'))

#### create model 5 (CNN) - relu

In [ ]:
# np.random.seed(777)

# model5 = Sequential()

# model5.add(Conv2D(filters = 32, kernel_size = (4,4), padding = 'Same', activation = 'relu', input_shape = (96, 96, 1)))
# model5.add(Conv2D(filters = 64, kernel_size = (4,4), padding = 'Same', activation = 'relu'))
# model5.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model5.add(Dropout(0.25))
# model5.add(Flatten())
# model5.add(Dense(256, activation = 'relu'))
# model5.add(Dropout(0.5))
# model5.add(Dense(128, activation = 'relu'))
# model5.add(Dropout(0.7))
# model5.add(Dense(30))

#### create model 6 (CNN) - elu + relu

In [ ]:
# np.random.seed(777)

# model6 = Sequential()

# model6.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'elu', input_shape = (96, 96, 1)))
# model6.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model6.add(Dropout(0.3))

# model6.add(Conv2D(filters = 32, kernel_size = (4,4), padding = 'Same', activation = 'elu'))
# model6.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model6.add(Dropout(0.5))

# model6.add(Flatten())
# model6.add(Dense(256, activation = 'elu'))
# model6.add(Dropout(0.5))
# model6.add(Dense(128, activation = 'relu'))
# model6.add(Dropout(0.7))
# model6.add(Dense(30))

#### create model 7 (CNN)

In [ ]:
# np.random.seed(777)

# model7 = Sequential()

# model7.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (96, 96, 1)))
# model7.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model7.add(Dropout(0.5))

# model7.add(Conv2D(filters = 32, kernel_size = (4,4), padding = 'Same', activation = 'relu'))
# model7.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model7.add(Dropout(0.5))
# model7.add(Flatten())

# model7.add(Dense(128, activation = 'relu'))
# model7.add(Dropout(0.7))
# model7.add(Dense(30))

#### create model 8 (CNN)

In [ ]:
# np.random.seed(777)

# model8 = Sequential()

# model8.add(Conv2D(filters = 64, kernel_size = (6,6), padding = 'Same', activation = 'relu', input_shape = (96, 96, 1)))
# model8.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model8.add(Dropout(0.3))

# model8.add(Conv2D(filters = 32, kernel_size = (4,4), padding = 'Same', activation = 'relu'))
# model8.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model8.add(Dropout(0.5))
# model8.add(Flatten())

# model8.add(Dense(256, activation = 'relu'))
# model8.add(Dropout(0.5))
# model8.add(Dense(128, activation = 'relu'))
# model8.add(Dense(30))

#### create model 9 (CNN)

In [ ]:
# np.random.seed(777)

# model9 = Sequential()

# model9.add(Conv2D(filters = 128, kernel_size = (5,5), padding = 'Same', activation = 'linear', input_shape = (96, 96, 1)))
# model9.add(LeakyReLU(alpha=.001))
# model9.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model9.add(Dropout(0.5))
# model9.add(Flatten())
# model9.add(Dense(256, activation = 'linear'))
# model9.add(LeakyReLU(alpha=.001))
# model9.add(Dropout(0.5))
# model9.add(Dense(128, activation = 'linear'))
# model9.add(LeakyReLU(alpha=.001))
# model9.add(Dropout(0.7))
# model9.add(Dense(30))

#### create model 10 (CNN) - LAST

In [ ]:
np.random.seed(777)

model10 = Sequential()

model10.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (96, 96, 1)))
model10.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model10.add(Dropout(0.3))

model10.add(Conv2D(filters = 32, kernel_size = (4,4), padding = 'Same', activation = 'relu'))
model10.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model10.add(Dropout(0.5))

model10.add(Flatten())
model10.add(Dense(128, activation = 'relu'))
model10.add(Dropout(0.7))
model10.add(Dense(30, activation = 'relu'))

#### Compile the model 1

In [ ]:
# model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae', 'accuracy'])

#### Compile the model 2

In [ ]:
# model2.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae', 'accuracy'])

#### Compile the model 3

In [ ]:
# model3.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae', 'accuracy'])

#### Compile the model 4

In [ ]:
# model4.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae', 'accuracy'])

#### Compile the model 5

In [ ]:
# model5.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

#### Compile the model 6

In [ ]:
# model6.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

#### Compile the model 7

In [ ]:
# model7.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

#### Compile the model 8

In [ ]:
# model8.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

#### Compile the model 9

In [ ]:
# model9.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

#### Compile the model 10

In [ ]:
model10.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])

### 모델 최적화 설정

In [ ]:
MODEL_DIR = '../model/'

In [ ]:
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [ ]:
modelpath = '../model/{epoch:02d}-{val_loss:4f}.hdf5'

In [ ]:
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

### go!

In [ ]:
# time1 = time.time()
# model.fit(x, y, epochs=100, batch_size=128, validation_split=0.2)
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# model2.fit(x, y, epochs=100, batch_size=100, validation_split=0.3)
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# model3.fit(x, y, epochs=100, batch_size=100, validation_split=0.2)
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# model4.fit(x, y, epochs=100, batch_size=100, validation_split=0.25)
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# history = model5.fit(x, y, validation_split=0.3, epochs=100, batch_size=100, verbose=1, callbacks=[early_stopping_callback, checkpointer])
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# history = model6.fit(x, y, validation_split=0.3, epochs=100, batch_size=100, verbose=1, callbacks=[early_stopping_callback, checkpointer])
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# history = model7.fit(x, y, validation_split=0.3, epochs=100, batch_size=100, verbose=1, callbacks=[early_stopping_callback, checkpointer])
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# history = model8.fit(x, y, validation_split=0.3, epochs=100, batch_size=100, verbose=1, callbacks=[early_stopping_callback, checkpointer])
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
# time1 = time.time()
# history = model9.fit(x, y, validation_split=0.2, epochs=100, batch_size=100, verbose=1, callbacks=[early_stopping_callback, checkpointer])
# time2 = time.time()
# print('Learning Finished!')
# chk_processting_time(time1, time2)

In [ ]:
time1 = time.time()
history = model10.fit(x, y, validation_split=0.2, epochs=100, batch_size=100, verbose=1, callbacks=[early_stopping_callback, checkpointer])
time2 = time.time()
print('Learning Finished!')
chk_processting_time(time1, time2)

### 모델에 예측값 넣기

In [ ]:
predict = model10.predict(x)

### loss 값 그래프로 출력 (train / validation)

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

x_len = np.arange(len(train_loss))
plt.plot(x_len, train_loss, marker='.', c='red', label='Train_loss')
plt.plot(x_len, val_loss, marker='.', c='blue', label='Val_loss')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

### 예측값 / 실제값 비교

In [ ]:
def result_show(x, y, predict):
    fig, axes = plt.subplots(1, 2, figsize=(10,5))
    
    for ax in axes:
        ax.imshow(x, 'gray')
        ax.axis('off')
        
    points = np.vstack(np.split(y, 15)).T
    p_points = np.vstack(np.split(predict, 15)).T
    
    axes[0].plot(p_points[0], p_points[1], 'o', color='red')
    axes[0].set_title('Predict_Keypoints', size=15)
    
    axes[1].plot(p_points[0], p_points[1], 'o', color='red')
    axes[1].plot(points[0], points[1], 'o', color='blue')
    axes[1].set_title('Result', size=15)

In [ ]:
sample_idx = np.random.choice(len(train))
result_show(train.Image[sample_idx], y[sample_idx], predict[sample_idx])

### Test data 예측하기

In [ ]:
x_t.shape

In [ ]:
y_t = model10.predict(x_t)

In [ ]:
sample_idx = np.random.choice(len(test))
keypoints_show(test.Image[sample_idx], y_t[sample_idx])

### 데이터 제출

In [ ]:
look_id = pd.read_csv('../input/IdLookupTable.csv')

In [ ]:
look_id.info()

In [ ]:
look_id.drop('Location', axis=1, inplace=True)

In [ ]:
look_id.info()

In [ ]:
ind = np.array(train.columns[:-1])
value = np.array(range(0,30))
maps = pd.Series(value, ind)

In [ ]:
look_id['location_id'] = look_id.FeatureName.map(maps)

In [ ]:
df = look_id.copy()

location = pd.DataFrame({'Location':[]})
for i in range(1,1784):
    ind = df[df.ImageId==i].location_id
    location = location.append(pd.DataFrame(y_t[i-1][list(ind)],columns=['Location']), ignore_index=True)

In [ ]:
look_id['Location']=location

In [ ]:
look_id[['RowId','Location']].to_csv('Predict.csv',index=False)

### Kaggle 답 제출

#### Kaggel API 설치

In [ ]:
# !pip install kaggle

In [ ]:
# !pip show kaggle

#### Kaggle API가 설치된 위치를 확인한 후 다운로드 한 kaggle.json 파일을 이동시킨다

In [ ]:
# !kaggle config path

#### Submission

In [ ]:
# ! kaggle competitions submit -c facial-keypoints-detection -f predict.csv -m'submission

In [ ]:
# !kaggle competitions submissions -c facial-keypoints-detection